In [1]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv',encoding='utf-8')
weibo_o = pd.read_csv('./data/Weibo.csv',encoding='utf-8')
wechat_o = pd.read_csv('./data/Wechat.csv',encoding='utf-8')
news_forum_o = pd.read_csv('./data/News_forum.csv',encoding='utf-8')
dianping = pd.read_csv('./data/dianping135.csv')
#dianping = pd.read_csv('./data/dianping_full.csv')

In [3]:
seed = 200
np.random.seed(seed)
cross_validation = 5

In [4]:
# dianping_pos = dianping[dianping['rate']>4]
# dianping_pos['SENTIMENT'] = 'Positive'
# dianping_neg = dianping[dianping['rate']<2]
# dianping_neg['SENTIMENT'] = 'Negative'
# dianping_neu = dianping[dianping['rate']==3]
# dianping_neu['SENTIMENT'] = 'Neutral'
# dianping = pd.concat([dianping_pos,dianping_neg,dianping_neu])

In [5]:
# dianping = dianping[['content','SENTIMENT']].dropna(axis=0, how = 'any')
# dianping.columns = ['CONTENT','SENTIMENT']
# dianping['CONTENT'] = dianping['CONTENT'].str.replace('\n', '')
# dianping = dianping.reset_index(drop=True)
# dianping.to_csv('./data/dianping.csv',encoding = 'utf-8',index=False)
# dianping = pd.read_csv('./data/dianping.csv')

In [6]:
dianping = dianping.reset_index(drop=True)
shuffle_indices = np.random.permutation(np.arange(len(dianping)))
dianping = dianping.loc[shuffle_indices]
dianping = pd.concat([dianping[dianping.SENTIMENT == 'Positive'][:10000],
                      dianping[dianping.SENTIMENT == 'Negative'][:20000],
                      dianping[dianping.SENTIMENT == 'Neutral'][:10000]
                    ])

# Sentiment data

In [17]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')
#news_forum_s = news_forum_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [18]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [19]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [20]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [21]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i])
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [22]:
if cross_validation is None:
    dev, test = train_test_split(data, test_size=0.2, random_state = seed)
else:
    kf = KFold(n_splits=cross_validation, shuffle = True, random_state = seed)

In [ ]:
train_folder = './data/processed/training/'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)
for the_file in os.listdir(train_folder):
    file_path = os.path.join(train_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
        
dianping_folder = './data/processed/training/dianping/'
if not os.path.exists(dianping_folder):
    os.makedirs(dianping_folder)
for the_file in os.listdir(dianping_folder):
    file_path = os.path.join(dianping_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
snownlp_folder = './data/processed/training/snownlp/'
if not os.path.exists(snownlp_folder):
    os.makedirs(snownlp_folder)
for the_file in os.listdir(snownlp_folder):
    file_path = os.path.join(snownlp_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [13]:
testing_folder = './data/processed/testing/'
if not os.path.exists(testing_folder):
    os.makedirs(testing_folder)
for the_file in os.listdir(testing_folder):
    file_path = os.path.join(testing_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)        
dev_folder = './data/processed/validation/'
if not os.path.exists(dev_folder):
    os.makedirs(dev_folder)
for the_file in os.listdir(dev_folder):
    file_path = os.path.join(dev_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

if cross_validation is not None:
    for i in range(cross_validation):
        foldername = testing_folder + 'folder' + str(i) + '/'
        if not os.path.exists(foldername):
            os.makedirs(foldername)
        for the_file in os.listdir(foldername):
            file_path = os.path.join(foldername, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                #elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)        
        foldername = dev_folder + 'folder' + str(i) + '/'
        if not os.path.exists(foldername):
            os.makedirs(foldername)
        for the_file in os.listdir(foldername):
            file_path = os.path.join(foldername, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                #elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)
        


In [19]:
# snownlp
shutil.copyfile("./data/pos.txt",snownlp_folder + "Positive.utf8")
shutil.copyfile("./data/neg.txt",snownlp_folder + "Negative.utf8")
os.mknod(snownlp_folder + "Neutral.utf8") 

In [20]:
# dianping
for x in dianping['SENTIMENT'].unique():
#     np.savetxt(dianping_folder+ x.replace('/',' ') +'.utf8',\
#                    dianping[dianping['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')
    filename = dianping_folder+ x.replace('/',' ') +'.utf8'
    os.mknod(filename)
    with open(filename,'w') as f:
        for l in dianping[dianping['SENTIMENT'] == x]['CONTENT'].values:
            f.write('%s\n'%l)


In [14]:
# manulife
if cross_validation is not None:
    i = 0
    for train_index, test_index in kf.split(data):
        dev, test = data.loc[train_index], data.loc[test_index]
        for x in dev['SENTIMENT'].unique():
            filename = dev_folder  + 'folder' + str(i) + '/' + x.replace('/',' ') +'.utf8'
            os.mknod(filename)
            with open(filename,'w') as f:
                for l in dev[dev['SENTIMENT'] == x]['CONTENT'].values:
                    f.write('%s\n'%l)

        for x in test['SENTIMENT'].unique():
            filename = testing_folder + 'folder' + str(i) + '/' + x.replace('/',' ') +'.utf8'
            os.mknod(filename)
            with open(filename,'w') as f:
                for l in test[test['SENTIMENT'] == x]['CONTENT'].values:
                    f.write('%s\n'%l)
        i += 1
else:
    for x in dev['SENTIMENT'].unique():
        filename = dev_folder+ x.replace('/',' ') +'.utf8'
        os.mknod(filename)
        with open(filename,'w') as f:
            for l in dev[dev['SENTIMENT'] == x]['CONTENT'].values:
                f.write('%s\n'%l)

    for x in test['SENTIMENT'].unique():
        filename = testing_folder+ x.replace('/',' ') +'.utf8'
        os.mknod(filename)
        with open(filename,'w') as f:
            for l in test[test['SENTIMENT'] == x]['CONTENT'].values:
                f.write('%s\n'%l)

In [15]:
dev['SENTIMENT'].value_counts()

Neutral     23113
Negative      875
Positive      729
Name: SENTIMENT, dtype: int64

In [16]:
test['SENTIMENT'].value_counts()

Neutral     5768
Negative     215
Positive     196
Name: SENTIMENT, dtype: int64

In [13]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model.all')

In [3]:
len(w2vModel.wv.vocab)

4806

In [275]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [16]:
vocab_obj = w2vModel.wv.vocab
word = '骗'
print(word)
print(vocab_obj[word].index)

骗
1266


In [41]:
dianping.head()

,CONTENT,SENTIMENT
0,仓储式购物环境，看着货架上一堆堆的物品，人的购物欲望就被激发起来了！进口食品很多，特别是西点...,Positive
1,有点远 在西安的时候逛得多我在岳麓区这边 每次去都要坐一个多小时的车 搞得后来没什么欲望去了...,Positive
2,实在要来赞一赞省博，首先网站做的很好，不多的能支持firefox的网站啊。网上订票系统非常靠...,Positive
3,第二次去了，正值周末，10点到已经只拿到下午2点的票子。估计排在后面的人是看不成了。马王堆是...,Positive
4,给予全五星的评价在于带我们的荣誉志愿者，唐老伯。呵呵，好像是财政厅退休公务员，带着湖南口音，...,Positive


In [70]:
from os import listdir

In [162]:
2. / (1. + np.exp(-10. * 0.01)) - 1

0.049958374957880025